# VizFlow Demo - v0.4.4

Demonstrates **separate trade/alpha presets** and **schema evolution** with real data.

- `trade_preset` for trade files (e.g., `ylin_v20251204`)
- `alpha_preset` for alpha files (e.g., `jyao_v20251114`)

**v0.4.4**: Column naming convention changed from `alpha_*` to `x_*` (ML-style)

In [ ]:
import polars as pl
import vizflow as vf
from pathlib import Path

print("VizFlow version:", vf.__version__)

VizFlow version: 0.4.3


## 1. Trade Data (ylin_v20251204 preset)

Load meords trade files with `trade_preset="ylin_v20251204"` - auto-renames 52 columns.

In [12]:
print("PRESETS:", vf.PRESETS.keys())

PRESETS: dict_keys(['ylin_v20251204', 'jyao_v20251114'])


In [7]:
config = vf.Config(
    trade_dir=Path("data/ylin/trade"),
    trade_pattern="{date}.meords",
    trade_preset="ylin_v20251204",
    market="CN",
    trade_schema={
        "order_qty": vf.ColumnSchema(cast_to=pl.Int64),
    },
)
vf.set_config(config)

df_trade = vf.scan_trade("11110101")
print("=== Trade Data (ylin_v20251204 preset) ===")
print(df_trade.select(['ukey', 'order_side', 'order_qty', 'bid_px0', 'ask_px0']).head(5).collect())

=== Trade Data (ylin_v20251204 preset) ===
shape: (5, 5)
┌──────────┬────────────┬───────────┬─────────┬─────────┐
│ ukey     ┆ order_side ┆ order_qty ┆ bid_px0 ┆ ask_px0 │
│ ---      ┆ ---        ┆ ---       ┆ ---     ┆ ---     │
│ i64      ┆ str        ┆ i64       ┆ f64     ┆ f64     │
╞══════════╪════════════╪═══════════╪═════════╪═════════╡
│ 11000408 ┆ Sell       ┆ 2900      ┆ 22.48   ┆ 22.5    │
│ 11002405 ┆ Buy        ┆ 27100     ┆ 7.19    ┆ 7.2     │
│ 11000739 ┆ Sell       ┆ 400       ┆ 14.97   ┆ 14.98   │
│ 11002709 ┆ Sell       ┆ 600       ┆ 14.7    ┆ 14.71   │
│ 11300436 ┆ Sell       ┆ 14000     ┆ 20.3    ┆ 20.31   │
└──────────┴────────────┴───────────┴─────────┴─────────┘


## 2. Alpha Data (jyao_v20251114 preset)

Load alpha files with `alpha_preset="jyao_v20251114"` - auto-renames alpha columns.

In [ ]:
config = vf.Config(
    alpha_dir=Path("data/jyao/alpha"),
    alpha_pattern="alpha_{date}.feather",
    alpha_preset="jyao_v20251114",
    market="CN",
)
vf.set_config(config)

df_alpha = vf.scan_alpha("20251114")
print("=== Alpha Data (jyao_v20251114 preset) ===")
print(df_alpha.select(['ukey', 'ticktime', 'bid_px0', 'ask_px0', 'x_10s', 'x_60s', 'x_3m', 'x_30m']).collect())

## 3. Parse Timestamps

In [4]:
df_alpha = vf.parse_time(df_alpha, timestamp_col="ticktime")
print("=== Parse Timestamps ===")
print(df_alpha.select(['ukey', 'ticktime', 'tod_ticktime', 'elapsed_ticktime']).head(6).collect())

=== Parse Timestamps ===
shape: (6, 4)
┌──────────┬──────────┬──────────────┬──────────────────┐
│ ukey     ┆ ticktime ┆ tod_ticktime ┆ elapsed_ticktime │
│ ---      ┆ ---      ┆ ---          ┆ ---              │
│ i64      ┆ i64      ┆ time         ┆ i64              │
╞══════════╪══════════╪══════════════╪══════════════════╡
│ 11000408 ┆ 93000000 ┆ 09:30:00     ┆ 0                │
│ 11000408 ┆ 93001000 ┆ 09:30:01     ┆ 1000             │
│ 11002405 ┆ 93000000 ┆ 09:30:00     ┆ 0                │
│ 11002405 ┆ 93001000 ┆ 09:30:01     ┆ 1000             │
│ 11000739 ┆ 93000000 ┆ 09:30:00     ┆ 0                │
│ 11000739 ┆ 93001000 ┆ 09:30:01     ┆ 1000             │
└──────────┴──────────┴──────────────┴──────────────────┘


## 4. Aggregation

In [ ]:
metrics = {
    "count": pl.len(),
    "avg_x_10s": pl.col("x_10s").mean(),
    "avg_x_3m": pl.col("x_3m").mean(),
}
agg_df = vf.aggregate(df_alpha, group_by=["ukey"], metrics=metrics)
print("=== Aggregation ===")
print(agg_df.collect())